# Perceptron Learning Algorithm (PLA)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
def determinant(X, f):
    """
    Determines the labels of X dataset by given f target function.
    Then, takes the sign of each label.
    """
    values = (f[1][0] - f[0][0])*(X[:, 2] - f[0][1])-(f[1][1] - f[0][1])*(X[:, 1] - f[0][0])
    return np.sign(values).astype(int)

def sign_determinant(X):
    """
    Finds the sign of each sample by the function provided in the question.
    """
    values = (X[:, 1]**2 + X[:, 2]**2 - 0.6)
    return np.sign(values).astype(int)

def create_data_and_pick_target_function(sample_size=10, feature_size=2, target_function=None):
    """
    Inputs:
        sample_size: size of the randomly generated data '10' as default
        feature_size: feature dimensions. '2' as default
        target_function: creates a random target function if not provided,
                         if a target function explicitly given, uses that
                         target function to create labels. Else: uses
                         sign_determinant function to generate labels.
    Outputs:
        X (dataset), f (target_function), and y (True labels)
    """
    X = np.random.uniform(-1, 1, (sample_size, feature_size))
    X = np.concatenate([np.ones((sample_size,1)), X], axis=1) # adds 1 to the left as X0
    f = target_function
    if target_function != 'sign_determinant':
        if target_function is None:
            f = np.random.uniform(-1, 1, (2, 2))
        y = determinant(X, f)
    else:
        y = sign_determinant(X)
    return X, f, y

In [8]:
class PLA:
    supported_weights = set(['zero', 'rand', 'custom'])

    def __init__(self, weight_type='zero', weights=None):
        """
        weight_type: either 'zero', 'rand' or custom to chose
                    zero or random initialization. Also you may
                    provide custom weights (requires weights parameter)
        weights:    custom weights for model (reqires weight_type='custom')
        """
        if weight_type not in self.supported_weights:
            raise ValueError(f"{weight_type} is not supported by parameter weight_type")

        self.weight_type = weight_type
        self.weights = weights

    def _initialize_weights(self, X):
        """
        initializes weights based on given type same as feature size
        """
        if self.weight_type == 'zero':
            self.w = np.zeros((1, X.shape[1]))
        elif self.weight_type == 'rand':
            self.w = np.random.rand((1, X.shape[1]))
        else:
            self.w = self.weights

    def update_weights(self, X, y):
        self.w += y * X

    def predict(self, X):
        """
        predict the output of given type
        """
        prob = np.ravel(np.dot(X, self.w.T))
        return np.sign(prob).astype(int)

    def fit(self, X, y):
        """
        fits the model using given dataset and corresponding targets
        """
        iteration = 1 # to count the number of iterations
        self._initialize_weights(X)
        while True:
            y_pred = self.predict(X) # predicts output to update weights
                                     # among misclassified points   
            if (y_pred == y).all():  # terminates the method if all matches
                return iteration
            
            indices = np.arange(X.shape[0]) # indices of all samples
            misclassified = y != y_pred     # to find misclassified points
            idx = np.random.choice(indices[misclassified]) # randomly choosing an index 
                                                           # of misclassified points 
            self.update_weights(X[idx], y[idx])
            iteration += 1

In [ ]:
class LinearRegression:
    supported_weights = set(['zero', 'rand'])

    def __init__(self, weight_type='zero'):
        """
        weigh_type: either 'zero' or 'rand' to chose
                    zero or random initialization
        """
        if weight_type not in self.supported_weights:
            raise ValueError(f"{weight_type} is not supported by parameter weight_type")

        self.weight_type = weight_type

    def _initialize_weights(self, X):
        if self.weight_type == 'zero':
            self.w = np.zeros((1, X.shape[1]))
        else:
            self.w = np.random.rand((1, X.shape[1]))

    def update_weights(self, X, y):
        X_inv = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
        self.w = np.dot(X_inv, y)

    def predict(self, X):
        return np.sign(np.dot(self.w, X.T))

    def fit(self, X, y):
        self._initialize_weights(X)
        self.update_weights(X, y)

# Questions

## The Perceptron Learning Algorithm

### Q4

In [9]:
iterations = [] # to store the # of iterations in each step
model = PLA()   # initializing a PLA model
errors = []
for i in range(1000):
    X, f, y = create_data_and_pick_target_function()
    X_test, _, y_test = create_data_and_pick_target_function(sample_size=1000, target_function=f)

    iteration = model.fit(X, y)
    y_pred = model.predict(X_test)

    iterations.append(iteration)
    errors.append(1 - accuracy_score(y_test, y_pred))

np.mean(iterations)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


10.378

### Q5

In [10]:
np.mean(errors)

0.10647200000000001

### Q6

In [11]:
iterations = []
model = PLA()
errors = []
for i in range(1000):
    X, f, y = create_data_and_pick_target_function(sample_size=100)
    X_test, _, y_test = create_data_and_pick_target_function(sample_size=1000, target_function=f)

    iteration = model.fit(X, y)
    y_pred = model.predict(X_test)

    iterations.append(iteration)
    errors.append(1 - accuracy_score(y_test, y_pred))

np.mean(iterations)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


97.897

### Q7

In [12]:
np.mean(errors)

0.013936000000000011

## Linear Regression

### Q8

In [ ]:
errors = []
model = LinearRegression()
for i in range(1000):
    X, f, y = create_data_and_pick_target_function(sample_size=100)
    model.fit(X, y)
    y_pred = model.predict(X)
    errors.append(1 - accuracy_score(y, y_pred))

np.mean(errors)

0.03847000000000001

### Q9

In [ ]:
errors = []
for i in range(1000):
    X, f, y = create_data_and_pick_target_function(sample_size=1000, target_function=f)
    y_pred = model.predict(X)
    errors.append(1 - accuracy_score(y, y_pred)) # error = 1 - accuracy

np.mean(errors)

0.007672000000000007

### Q10

In [ ]:
reg_model_w = model.w
iterations = []
model = PLA(weight_type='custom', weights=reg_model_w)
for i in range(1000):
    X, f, y = create_data_and_pick_target_function(sample_size=10, target_function=f)
    iteration, error = model.fit(X, y)
    iterations.append(iteration)

np.mean(iterations)

1.439

## Nonlinear Transformation

### Q11

In [ ]:
def flip_y(y):
    rand_choices = np.random.choice(np.arange(y.shape[0]), 100)
    y[rand_choices] *= -1
    return y

In [ ]:
errors = []
model = LinearRegression()
for i in range(1000):
    X, f, y = create_data_and_pick_target_function(1000, target_function="sign_determinant")
    y = flip_y(y)
    model.fit(X, y)
    y_pred = model.predict(X)
    errors.append(1 - accuracy_score(y, y_pred))

np.mean(errors)

0.506501

### Q12

In [ ]:
model = LinearRegression()

X, f, y = create_data_and_pick_target_function(sample_size=1000, target_function="sign_determinant")
X = np.concatenate([X, 
                    (X[:, 1]*X[:, 2]).reshape(-1,1), 
                    (X[:, 1]**2).reshape(-1,1), 
                    (X[:, 2]**2).reshape(-1,1)], axis=1) # concatenating all features
y = flip_y(y)
model.fit(X, y)

model.w

array([-1.0206148 , -0.04560406,  0.00526564, -0.01008764,  1.50361943,
        1.64873634])

### Q13

In [ ]:
errors = []
for i in range(1000):
    X, f, y = create_data_and_pick_target_function(sample_size=1000, target_function="sign_determinant")
    X = np.concatenate([X, 
                        (X[:, 1]*X[:, 2]).reshape(-1,1), 
                        (X[:, 1]**2).reshape(-1,1), 
                        (X[:, 2]**2).reshape(-1,1)], axis=1)
    y = flip_y(y)
    y_pred = model.predict(X)
    errors.append(1 - accuracy_score(y, y_pred))

np.mean(errors)

0.12599000000000002